In [ ]:
using Images, Colors

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho)  #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto) #el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 0:alto-1, x in 0:ancho-1]
end

# Algoritmo de escape (trampas)

Ahora bien, para hacer un dibujito dinámico debemos asignar a cada número complejo del muestreo un color que dependa de las propiedades dinámicas de dicho numero.

Una **trampa** o **algoritmo de escape** consiste en una región del plano complejo $U \subset \mathbb{C}$, una sucesión de colores $\{ c_i \}$ y un mapa $f: \mathbb{C} \rightarrow \mathbb{C}$.

Para un número complejo $z$ el color que se elige es $c_n$, donde $n$ es el mínimo tal que $f^{n} (z) \in U$.

En la práctica es necesario limitar la cantidad de iteraciones permitidas y asignar un color a los puntos que no llegan a la región $U$.

Asimismo, en la práctica las regiones que conforman las trampas deben ser *computacionalmente sencillas*. En general, evaluar una sentencia del tipo $z \in U$ puede ser complicado o incluso imposible. Las regiones que usaremos en casi todos los ejemplos serán discos de manera que la pertenencia se reduce a la evaluacion de una sentencia de tipo $|z - w| < R$. 

## Conjuntos de Julia Rellenos

Sabemos de nuestro curso introductorio a la dinámica holomorfa que todo polinomio de grado $\geq 2$ tiene a $\infty$ como un punto fijo super-atractor, por lo que podemos encontrar un disco suficientemente grande tal que su complemento esta completamente contenido dentro de la cuenca de atracción de $\infty$, llamemos al radio de este disco ``R``.

El complememto de este disco contiene un dominio fundamental de la cuenca de atracción de $\infty$, es decir, la órbita de todo punto que eventualmente se vea atraido a $\infty$ intersecta a esta región.

Una manera de pintar el dibujito de la cuenca de atracción de $\infty$ para polinomios es usando una trampa en los valores del muestreo. La región de la trampa será el complemento de un disco centrado en el origen de radio ``R`` y el mapa sea el polinomio mismo.

La *paleta de colores* sera como la del ejemplo del ``argumento_complejo``, variaremos el tono del color dependiendo del *tiempo de escape*.

In [ ]:
function mi_primer_trampa(Z, F, R=5.0, TOPE=1000) #regresa el color segun el tiempo que toma salir del disco de radio R 
    cont = 0
    while cont <= TOPE
        if(abs(Z) > R) 
            return HSV{Float64}((17.6180339887498948482045868343*cont)%360.0,1.0,1.0)
        end
        Z = F(Z)
        cont = cont + 1
    end
    return HSV{Float64}(0.0, 0.0, 0.0) #color negro
end

La función ``mi_primer_trampa`` toma como parametros el punto que se va a evaluar ``Z``, el mapa a iterar ``F``, el radio de escape ``R``  y el ``TOPE`` a la cantidad de iteraciones.

Ahora aplicaremos esta función usando broadcast a un arreglo de numeros proporcionado por nuestra ``muestra``.

In [ ]:
function julia_relleno(POLINOMIO, R=5.0, TOPE=1000, centro=0.0, radio=2.0, ancho=1000, alto=1000)
    broadcast(Z -> mi_primer_trampa(Z, POLINOMIO, R, TOPE), muestra(radio, centro, alto, ancho))
end

Veamos unos ejemplos:

In [ ]:
mi_polinomio(z)=z*z*z + (0.48735im - 0.49864)*z + (0.3198541 + 0.67546im)

julia_relleno(mi_polinomio)

In [ ]:
julia_relleno(mi_polinomio, 5.0, 1000, 0.4im, 0.5)

In [ ]:
julia_relleno(mi_polinomio, 5.0, 1000, 0.07+0.42im, 0.05)

## Espacios de parametros: El conjunto de Mandelbrot

Nos recordamos que el conjunto de Mandelbrot consiste de los numeros complejos $c$ tales que la órbita del $0$ no escapa a infinito bajo la iteración del mapa $f(z) = z^2 + c$.

Notese que para hacer el dibujito podemos aplicar una trampa al valor $0$ con diferentes mapas para cada valor de $c$ en una muestra y la misma región de escape para todos los parametros.

In [ ]:
function mandelbrot(R=3.0, TOPE=1000, centro=0.0, radio=2.0, ancho=1000, alto=1000)
    broadcast(C -> mi_primer_trampa(0.0, z -> z^2 + C, R, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
mandelbrot()

In [ ]:
mandelbrot(2.0, 500, -0.75, 1.4)